In [32]:
import os
import tqdm

raw_data="/data/nas/ASR_Datasets/data_aishell2/"

train_data_path=os.path.join(raw_data,"data_aishell2")
train_wav_path=os.path.join(train_data_path,"wav")
train_txt_path=os.path.join(train_data_path,"transcript/trans.txt")

outpath="/ssd/zhuang/code/FunASR2024_for_debug/examples/aishell2/DATA/mydata"
os.makedirs(outpath,exist_ok=True)

### 读取train  wav文件

In [2]:
train_wav_list=[]
for root, dirs, files in os.walk(train_wav_path):
    for file in files:
        if file.endswith(".wav"):
            full_path = os.path.join(root, file)
            train_wav_list.append(full_path)
print ("Done! Find the train wav length is",len(train_wav_list))

Done! Find the train wav length is 1009223


### 构建id2wav字典

In [21]:
id2wav = {}
for wav in train_wav_list:
    id = wav.split("/")[-1].split(".")[0]
    id2wav[id] = wav
print ("Done!")

Done!


### 读取train txt文件

In [20]:
id2text = {}
with open(train_txt_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    lines = [line.strip() for line in lines]  # 清理每行的首尾空白字符
for l in lines:
    id, txt = l.strip().split("	")
    id2text[id] = txt
print ("Done! Find the train txt length is",len(lines))

Done! Find the train txt length is 1009223


### 配对id-wav-text

In [28]:
import re

def contains_non_chinese(text):
    # 正则表达式匹配非中文字符
    non_chinese_pattern = re.compile(r'[^\u4e00-\u9fff]')
    return non_chinese_pattern.search(text) is not None

In [30]:
id2text_clean = {}
id2wav_clean = {}
for id, txt in id2text.items():
    if contains_non_chinese(txt):
        continue
    else:
        id2text_clean[id] = txt
        id2wav_clean[id] = id2wav[id]

id2text = id2text_clean
id2wav = id2wav_clean
if len(id2text) != len(id2wav):
    print("Error: the number of wav files and text files is not equal!")
    exit(1)
else:
    print ("Done! Find the train txt length is",len(id2text))

Done! Find the train txt length is 962818


In [34]:
train = os.path.join(outpath, "train")
os.makedirs(train, exist_ok=True)

trian_text = os.path.join(train, "text")
trian_wav_scp = os.path.join(train, "wav.scp")

with open(trian_text, 'w', encoding='utf-8') as file:
    for id, txt in id2text.items():
        file.write(id + " " + txt + "\n")

with open(trian_wav_scp, 'w', encoding='utf-8') as file:
    for id, wav in id2wav.items():
        file.write(id + " " + wav + "\n")
    
print ("Done!")

Done!


In [25]:
len(id2text) == len(id2wav)

True

### dev&test

In [35]:
dt_raw_data="/data/nas/ASR_Datasets/data_aishell2/devNtest/"

device_list=["Android","IOS","MIC"]
type_list=["dev","test"]

outpath="/ssd/zhuang/code/FunASR2024_for_debug/examples/aishell2/DATA/mydata"
os.makedirs(outpath,exist_ok=True)

In [ ]:
for device in device_list:
    for type in type_list:
        data_path=os.path.join(dt_raw_data,device,type)
        data_wav_path=os.path.join(data_path,"wav")
        data_txt_path=os.path.join(data_path